##To display this dashboard in your browser, please cd to the directory in which the notebook is contained on your machine and type in  "Voila Dashboard_Smartmeters --theme=dark"

In [2]:
from pandas import datetime
import datetime as dt
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from math import sqrt
import math
import pendulum
from fbprophet import Prophet
import logging
import ipywidgets as wg
from IPython.display import display
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

Importing plotly failed. Interactive plots will not work.


In [31]:
df = []
for j in range(0,112):
    df.append(pd.read_csv("/Users/user/Documents/Stutern Task_files/Final_Power_Project/Smart_Meters/daily_dataset/daily_dataset/block_"+str(j)+".csv"))    
energy = pd.concat(df, ignore_index = True)

In [4]:
def plot_usage (Address, Enquiry_type, Year, Month, Week):
    #Address = 'MAC000002'#str(input("Please input your adress: "))
    #Enquiry_type = 'Annual'#str(input("What kind of summary do you want? Weekly, Monthly, or Annual? "))
    #Year = '13'#str(input("Year? 12 13 14: "))
    #Month = '04'#str(input("Month? 01 02 03 04 05 06... 12: "))
    #Week = 3#int(input("Week? 1 2 3 4 5: "))
    
    House_02 = energy[energy['LCLid']==Address]
    House_02=House_02[['day', 'energy_sum']]
    House_02.day = pd.to_datetime(House_02.day, format = '%Y-%m-%d').dt.date
    House_02 = House_02.reset_index()
    House_02.drop('index', axis = 1, inplace = True)
    House_02['week_number']=0
    House_02['day_number']=0
    
    for j in range(len(House_02.day)):
        House_02['week_number'][j] = House_02.day[j].isocalendar()[1]
        House_02['day_number'][j] = House_02.day[j].isocalendar()[2]
    House_02['month']=House_02['day'].apply(lambda x: x.strftime('%m'))
    House_02['week_days']=House_02['day'].apply(lambda x: x.strftime('%A'))
    House_02['year']=House_02['day'].apply(lambda x: x.strftime('%y'))
    
    for j in range(len(House_02)):
        dt = pendulum.parse(str(House_02.day[j]))
        House_02.week_number[j] = dt.week_of_month
        
    if Enquiry_type == 'Annual':
        sns.set_style('darkgrid')
        figsize = (30,5)
        sns.barplot(data = House_02, x = 'year', y = 'energy_sum')
        plt.ylabel('Electricity Consumed')
        
    if Enquiry_type == 'Monthly':
        sns.set_style('darkgrid')
        figsize = (30,5)
        sns.barplot(data = House_02[House_02['year']==Year], x = 'month', y = 'energy_sum')
        plt.ylabel('Electricity Consumed')
        
    elif Enquiry_type == 'Weekly':
        sns.set_style('darkgrid')
        figsize = (30,5)
        sns.barplot(data = House_02[House_02['month']==Month], x = 'week_number', y = 'energy_sum')
        plt.ylabel('Electricity Consumed')
    
    elif Enquiry_type == 'Daily':
        sns.set_style('darkgrid')
        figsize = (30,5)
        sns.barplot(data = House_02[House_02['week_number']==Week], x = 'week_days', y = 'energy_sum')
        plt.ylabel('Electricity Consumed')

In [5]:
print("Hi! Thank you for patronizing us. You can quickly visualize your electricity usage here by choosing your address and selecting the appropriate categories in the dropdown menus")

Hi! Thank you for patronizing us. You can quickly visualize your energy usage here by choosing your address and selecting the appropriate categories in the dropdown menus


In [6]:
#plot_usage('MAC000002', 'Annual', '13', '4', 3)
Address_s = wg.Dropdown(options = list(energy['LCLid'].unique()), value = 'MAC000002')
Enquiry_type_s = wg.Dropdown(options=[('Annual', 'Annual'), ('Monthly', 'Monthly'), ('Weekly', 'Weekly'), ('Daily', 'Daily')], value = 'Annual')
Year_s = wg.Dropdown(options = ['12', '13', '14'], value = '13')
Month_s = wg.Dropdown(options = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12'], value = '04')
Week_s = wg.Dropdown(options = [1, 2, 3, 4, 5], value = 3)

wg.interact(plot_usage, Address=Address_s, Enquiry_type=Enquiry_type_s, Year=Year_s, Month=Month_s,Week=Week_s)

interactive(children=(Dropdown(description='Address', options=('MAC000002', 'MAC000246', 'MAC000450', 'MAC0010…

<function __main__.plot_usage(Address, Enquiry_type, Year, Month, Week)>

In [1]:
# energy_FBP = energy[['day', 'energy_sum']]
# energy_FBP.day = pd.to_datetime(energy_FBP.day, format = '%Y-%m-%d').dt.date
# energy_FBP.columns = ['ds', 'y']
# fb_instance = Prophet()
# fb_instance.fit(energy_FBP)


# def fb_predict(prediction_size, Address):
#     to_predict = energy[energy['LCLid']==Address]
#     to_predict = to_predict[['day', 'energy_sum']]
#     to_predict.columns = ['ds', 'y']
#     future_df = fb_instance.make_future_dataframe(periods = prediction_size)
#     forecast = fb_instance.predict(future_df)
#     return forecast

# prediction_size_s = wg.IntSlider(value = 30, min = 15, max = 45, step = 5)
# wg.interact(fb_predict, Address=Address_s, prediction_size = prediction_size_s)